In [58]:
# Install necessary libraries
!pip install pytesseract Pillow requests langchain_community langchain_core langchain_huggingface llama-index

import pytesseract
from PIL import Image
import os
from langchain_community.vectorstores import Chroma
from langchain_community.chat_models import ChatOllama
from langchain.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder
)
from langchain.chains import create_history_aware_retriever
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.messages import HumanMessage
from langchain_huggingface import HuggingFaceEmbeddings
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader

from langchain_text_splitters import RecursiveCharacterTextSplitter

# Load the image of the income statement
image_path = './data/is2.png'
image = Image.open(image_path)

# Use Tesseract to extract text from the image
extracted_text = pytesseract.image_to_string(image)
print("Extracted Text:")
print(extracted_text)

# Parsing the extracted text (This part depends on the format of your income statement)
# For demonstration, let's assume the text is formatted in a simple way

# def parse_income_statement(text):
#     lines = text.split('\n')
#     income_statement = {}
#
#     for line in lines:
#         print(line)
#         if 'Revenue' in line:
#             income_statement['Revenue'] = float(line.split()[-1].replace(',', ''))
#         elif 'Cost of Goods Sold' in line:
#             income_statement['Cost of Goods Sold'] = float(line.split()[-1].replace(',', ''))
#         elif 'Gross Profit' in line:
#             income_statement['Gross Profit'] = float(line.split()[-1].replace(',', ''))
#         elif 'Operating Expenses' in line:
#             income_statement['Operating Expenses'] = float(line.split()[-1].replace(',', ''))
#         elif 'Net Income' in line:
#             income_statement['Net Income'] = float(line.split()[-1].replace(',', ''))
#
#     return income_statement
#
# parsed_data = parse_income_statement(extracted_text)
# print("Parsed Data:")
# print(parsed_data)

  Obtaining dependency information for llama-index from https://files.pythonhosted.org/packages/8a/7c/0591ad0ba69b288a3531220d72ee51c2796209e34dd1a2c3c28a242ff59b/llama_index-0.10.61-py3-none-any.whl.metadata
  Obtaining dependency information for llama-index-agent-openai<0.3.0,>=0.1.4 from https://files.pythonhosted.org/packages/c2/9a/9a327ff664e4904b6806f716bf705041d3015b99b12568872833df10b18f/llama_index_agent_openai-0.2.9-py3-none-any.whl.metadata
  Obtaining dependency information for llama-index-cli<0.2.0,>=0.1.2 from https://files.pythonhosted.org/packages/fc/74/58a8f8b33bc709947ed08f29055967f49efa995aac59f45d7c4443814d0d/llama_index_cli-0.1.13-py3-none-any.whl.metadata
  Obtaining dependency information for llama-index-core==0.10.61 from https://files.pythonhosted.org/packages/69/03/4a925e779805dcc0332283960ffce4d9573b71109b2794f8a9294efdea05/llama_index_core-0.10.61-py3-none-any.whl.metadata
  Obtaining dependency information for llama-index-embeddings-openai<0.2.0,>=0.1.5 fro

In [59]:
local_llm = 'llama3'
llm = ChatOllama(
    model=local_llm,
    format="json",
    temperature=0
)

print(extracted_text)
documents = SimpleDirectoryReader("data").load_data()



embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = Chroma.from_documents(
    documents=documents,
    collection_name="rag-chroma",
    embedding=embedding,
)

retriever = vectorstore.as_retriever()

contextualize_q_system_prompt = """can you analyze this? only 2015 data"""
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)

AMAZON.COM, INC.

CONSOLIDATED STATEMENTS OF OPERATIONS
(in millions, except per share data)

‘Year Ended December 31,

2015 2016 2017
Net product sales $ 79,268 $ 94,665 §$ 118,573
Net service sales 27,738 41,322 59,293
Total net sales ~~ 107,006 135,987 177,866
Operating expenses:
Cost of sales 71,651 88,265 111,934
Fulfillment 13,410 17,619 25,249
Marketing 5,254 7,233 10,069
Technology and content 12,540 16,085 22,620
General and administrative 1,747 2,432 3,674
Other operating expense, net 171 167 214
Total operating expenses 104,773 131,801 173,760
Operating income 3933 4,186 4,106
Interest income 50 100 202
Interest expense (459) (484) (848)
Other income (expense), net (256) 90 346
Total non-operating income (expense) (665) (294) (300)
Income before income taxes 1,568 3,892 3,806
Provision for income taxes (950) (1,425) (769)
Equity-method investment activity, net of tax (22) (96) (4)
Net income S596 8 2371 $ 3,033
Basic earnings per share S$ 128 S$ 501 $ 632
Diluted earnings pe

AttributeError: 'ImageDocument' object has no attribute 'page_content'

In [ ]:

qa_system_prompt = """system You are an assistant for question-answering tasks. Use the following context to answer the question. Avoid phrases like "Based on the provided context". Explain the answer in the end. and make a heading with paragraph.
Question: {input}
Context: {context}
Answer: assistant"""
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)
chat_history = []

question = "Analyze this income statement."
ai_msg_1 = rag_chain.invoke({"input": question, "chat_history": chat_history})
chat_history.extend([HumanMessage(content=question), ai_msg_1["answer"]])
print(ai_msg_1["answer"])
